In [12]:
import pandas as pd
import numpy as np

In [13]:
# Check if running in Google Colab
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    # Define the base path for your data directory for Colab
    basePath = '/content/drive/MyDrive'

else:
    # Running locally or in a different environment
    basePath = '..'

In [14]:
# 1. Load the CSV file
file_path = 'data/processed/unified_countries.csv'

df = pd.read_csv(f'{basePath}/{file_path}')
df.head(3)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,...,PRCP_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES,Source_Station_ID,Source_Country_Name,Source_Station_Name
0,TS000060765,"GABES, TS",33.88,10.1,4.0,1887,NaN,NaN,NaN,NaN,...,",E",NaN,NaN,NaN,NaN,12.2,E,TS000060765,Tunisia,GABES
1,TS000060765,"GABES, TS",33.88,10.1,4.0,1889,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.7,E,TS000060765,Tunisia,GABES
2,TS000060765,"GABES, TS",33.88,10.1,4.0,1907,NaN,NaN,NaN,NaN,...,",E",NaN,NaN,NaN,NaN,NaN,NaN,TS000060765,Tunisia,GABES


In [15]:
# 3. Remove duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

In [16]:
# 4. Convert applicable columns to numeric (safely)
# Skip non-numeric columns like 'STATION', 'NAME', 'DATE', 'Source_*'
non_numeric_cols = ['STATION', 'NAME', 'DATE', 'Source_Station_ID', 'Source_Country_Name', 'Source_Station_Name']
numeric_cols = [col for col in df.columns if col not in non_numeric_cols]

In [17]:
# Convert with errors='coerce' to handle any invalid strings
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [18]:
# 5. Rearrange columns for better structure
reordered_cols = (
    ['Source_Country_Name', 'Source_Station_ID', 'Source_Station_Name',
     'STATION', 'NAME', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']
    + sorted([col for col in df.columns if col not in [
        'Source_Country_Name', 'Source_Station_ID', 'Source_Station_Name',
        'STATION', 'NAME', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']])
)
df = df[reordered_cols]

In [19]:
df.head(3)

,Source_Country_Name,Source_Station_ID,Source_Station_Name,STATION,NAME,DATE,LATITUDE,LONGITUDE,ELEVATION,CDSD,...,HTDD,HTDD_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
0,Tunisia,TS000060765,GABES,TS000060765,"GABES, TS",1887,33.88,10.1,4.0,NaN,...,NaN,NaN,134.7,NaN,NaN,NaN,NaN,NaN,12.2,NaN
1,Tunisia,TS000060765,GABES,TS000060765,"GABES, TS",1889,33.88,10.1,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.7,NaN
2,Tunisia,TS000060765,GABES,TS000060765,"GABES, TS",1907,33.88,10.1,4.0,NaN,...,NaN,NaN,109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Renaming for clarity first
df = df.rename(columns={
    'Source_Country_Name': 'COUNTRY',
    'NAME': 'STATION_NAME'
})

df = df.drop(columns=['Source_Station_ID', 'Source_Station_Name'])

In [21]:
# 2. Remove columns with more than 80% missing values
threshold = 0.8
missing_ratio = df.isnull().mean()
df = df.loc[:, missing_ratio <= threshold]

In [22]:
# 6. Save cleaned data
output_path = 'data/processed/cleaned_data1.csv'

df.to_csv(f'{basePath}/{output_path}', index=False)

print(f"Cleaning complete. Saved as '{output_path}'")

Cleaning complete. Saved as 'data/processed/cleaned_data1.csv'
